# imports and setup

In [1]:
import os
import json
import requests
import pandas as pd
from datetime import datetime
from ipywidgets import interact
# from widgetsnbextension import interact

c:\Users\ALAM\miniconda3\envs\data\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:

pd.options.plotting.backend = "plotly"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)


In [7]:

# PROJECT_DIRECTORY = os.getcwd()
PROJECT_DIRECTORY = os.path.normpath(os.getcwd())
# DATA_DIRECTORY = PROJECT_DIRECTORY + '\\data\\'
DATA_DIRECTORY = os.path.normpath(os.getcwd() + os.sep + os.pardir + os.sep + 'data')
# OUT_DIRECTORY = PROJECT_DIRECTORY + '\\out\\'
OUT_DIRECTORY = os.path.normpath(os.getcwd() + os.sep  + os.pardir  + os.sep + 'out')


In [8]:
print("PROJECT_DIRECTORY", PROJECT_DIRECTORY)
print("DATA_DIRECTORY", DATA_DIRECTORY)
print("OUT_DIRECTORY", OUT_DIRECTORY)

PROJECT_DIRECTORY c:\locr\mendel\notebooks
DATA_DIRECTORY c:\locr\mendel\data
OUT_DIRECTORY c:\locr\mendel\out


---

# load & transform data

In [9]:

def parse_date(date_string):
    """Parse an ISO 8601 formatted date string to a datetime object."""
    if date_string:
        return datetime.fromisoformat(date_string.replace('Z', '+00:00'))
    return None

def format_date(date_string: str):
    """Parse an ISO 8601 formatted date string to a datetime object."""
    if date_string:
        return (date_string[:4] + '-' + date_string[4:6] + '-' + date_string[6:])
    return None


In [10]:

def extract_data_from_json(file_path):
    """Extract patient, medication, and encounter data from a FHIR JSON file."""
    print("processing: ", file_path)
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    patient_info = {}
    medication_requests = []
    encounter_requests = []

    for entry in data.get('entry', []):
        resource = entry.get('resource', {})
        resource_type = resource.get('resourceType')

        if resource_type == 'Patient':
            patient_info = {
                'id': resource.get('id'),
                'gender': resource.get('gender'),
                'birthDate': parse_date(resource.get('birthDate')),
                'name': resource.get('name', [{}])[0].get('text', '')
            }

        elif resource_type == 'MedicationRequest':
            medication_requests.append({
                'id': resource.get('id'),
                'patient_id': resource.get('subject', {}).get('reference', '').split('/')[-1],
                'status': resource.get('status'),
                'intent': resource.get('intent'),
                'medication': resource.get('medicationCodeableConcept', {}).get('text'),
                'code': resource.get('medicationCodeableConcept', {}).get('coding', [{}])[0].get('code'),
                'display_name': resource.get('medicationCodeableConcept', {}).get('coding', [{}])[0].get('display'),
                'reason': resource.get('reasonReference', [{}])[0].get('display'),
                'authoredOn': parse_date(resource.get('authoredOn'))
            })

        elif resource_type == 'Encounter':
            encounter_requests.append({
                'id': resource.get('id'),
                'patient_id': resource.get('subject', {}).get('reference', '').split('/')[-1],
                'status': resource.get('status'),
                'class': resource.get('class', {}).get('code'),
                'type': resource.get('type', [{}])[0].get('text', ''),
                # 'display': resource.get('type', [{}])[0].get('coding', [{}])[0].get('display'),
                'start': parse_date(resource.get('period', {}).get('start')),
                'end': parse_date(resource.get('period', {}).get('end'))
            })

    return patient_info, medication_requests, encounter_requests


In [11]:

def validate_fhir_json(json_data):
                """Validate that JSON follows basic FHIR resource format requirements."""
                if not isinstance(json_data, dict):
                    return False
                
                # Check required FHIR elements
                if 'resourceType' not in json_data:
                    return False
                    
                # Check entry array exists and contains resources
                if 'entry' not in json_data:
                    return False
                    
                entries = json_data.get('entry', [])
                if not isinstance(entries, list):
                    return False
                    
                # Validate each entry has resource and resourceType
                for entry in entries:
                    if not isinstance(entry, dict):
                        return False
                    if 'resource' not in entry:
                        return False
                    if 'resourceType' not in entry['resource']:
                        return False
                        
                return True


In [12]:

def process_json_files(directory):
    """Process all JSON files in the given directory and extract patient, medication, and encounter data."""
    all_patients = []
    all_medication_requests = []
    all_encounter_requests = []

    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)

            with open(file_path) as f:
                print(f"checking: {filename}")
                json_data = json.load(f)

            if not validate_fhir_json(json_data):
                print(f"Invalid FHIR JSON format in file: {filename}")
                continue

            patient, medication_requests, encounters_requests = extract_data_from_json(file_path)

            if patient:
                all_patients.append(patient)
            all_medication_requests.extend(medication_requests)
            all_encounter_requests.extend(encounters_requests)

    return all_patients, all_medication_requests, all_encounter_requests


In [15]:
DATA_DIRECTORY = 'c:\\locr\\mendel\\data'

try:
    print(DATA_DIRECTORY)
    if not os.path.exists(DATA_DIRECTORY):
        os.makedirs(DATA_DIRECTORY)
        print("path does not exist")
    for filename in os.listdir(DATA_DIRECTORY):
        if filename.endswith('.json'):
            file_path = os.path.join(DATA_DIRECTORY, filename)
            print(file_path)
except OSError as e:
    print(f"Error accessing directory: {e}")


c:\locr\mendel\data
c:\locr\mendel\data\Ali918_invalid.json
c:\locr\mendel\data\Ali918_Klocko335_c16b9aea-2b5f-3866-22a1-01dea645c9e1.json
c:\locr\mendel\data\Alvin56_Rempel203_3a46995f-0d6d-db33-42b9-1582414dffef.json
c:\locr\mendel\data\Barry322_Balistreri607_ad634079-c15f-8a99-31a5-f9427a1889a4.json
c:\locr\mendel\data\Barry322_Langworth352_b9c319c3-5a7d-69a7-ba09-368625f2bb88.json
c:\locr\mendel\data\label.search=1535362.json
c:\locr\mendel\data\label.search=197591.json
c:\locr\mendel\data\search=active_ingredient-acetaminophen.json


In [16]:

patients, medications, encounters = process_json_files('c:\locr\mendel\data')

patients_df = pd.DataFrame(patients)
medication_requests_df = pd.DataFrame(medications)
encounters_df = pd.DataFrame(encounters)


checking: Ali918_invalid.json
Invalid FHIR JSON format in file: Ali918_invalid.json
checking: Ali918_Klocko335_c16b9aea-2b5f-3866-22a1-01dea645c9e1.json
processing:  c:\locr\mendel\data\Ali918_Klocko335_c16b9aea-2b5f-3866-22a1-01dea645c9e1.json
checking: Alvin56_Rempel203_3a46995f-0d6d-db33-42b9-1582414dffef.json
processing:  c:\locr\mendel\data\Alvin56_Rempel203_3a46995f-0d6d-db33-42b9-1582414dffef.json
checking: Barry322_Balistreri607_ad634079-c15f-8a99-31a5-f9427a1889a4.json
processing:  c:\locr\mendel\data\Barry322_Balistreri607_ad634079-c15f-8a99-31a5-f9427a1889a4.json
checking: Barry322_Langworth352_b9c319c3-5a7d-69a7-ba09-368625f2bb88.json
processing:  c:\locr\mendel\data\Barry322_Langworth352_b9c319c3-5a7d-69a7-ba09-368625f2bb88.json
checking: label.search=1535362.json
Invalid FHIR JSON format in file: label.search=1535362.json
checking: label.search=197591.json
Invalid FHIR JSON format in file: label.search=197591.json
checking: search=active_ingredient-acetaminophen.json
Inva

In [19]:
from ipywidgets import interact
pd.options.plotting.backend = "plotly"

# medication_requests_df
# encounters_df
# patients_df


In [27]:
patients_df.head(10)

,id,gender,birthDate,name
0,c16b9aea-2b5f-3866-22a1-01dea645c9e1,male,1991-05-07,
1,3a46995f-0d6d-db33-42b9-1582414dffef,male,2013-08-01,
2,ad634079-c15f-8a99-31a5-f9427a1889a4,male,1981-05-01,
3,b9c319c3-5a7d-69a7-ba09-368625f2bb88,male,2000-03-14,


In [46]:
@interact
# def filter_by_gender(df=patients_df['gender'], condition='male'):
def filter_by_gender(df=patients_df, column='gender', condition='male'):
    """Filter the data by a column value condition."""
    if isinstance(df, str):
        return "Please select a DataFrame column"
    filtered_data = df[df[column] == condition]
    print(f"Filtered data: {filtered_data.shape[0]} rows")
    return filtered_data.head()

# fdf = filter_by_gender(patients_df)
# fdf

interactive(children=(Dropdown(description='df', options=('id', 'gender', 'birthDate', 'name'), value='id'), T…

In [17]:

# df_patients.to_csv('patients.csv', index=False)
# df_medication_requests.to_csv('medication_requests.csv', index=False)
# df_encounters.to_csv('encounters.csv', index=False)


---

# query

In [50]:
FHIR_product_code = "1535362"  

In [51]:

def search_fda_drugs(drug_code):
    url = f"https://api.fda.gov/drug/label.json?search={drug_code}"
    response = requests.get(url, timeout=5)
    if response.status_code == 200:
        drug_data = response.json()
        print(drug_data)
        return drug_data
    else:
        print(f"Error: {response.status_code}")


In [52]:

drugs_json = search_fda_drugs(FHIR_product_code)
drugs_json


{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.', 'terms': 'https://open.fda.gov/terms/', 'license': 'https://open.fda.gov/license/', 'last_updated': '2024-11-01', 'results': {'skip': 0, 'limit': 1, 'total': 21}}, 'results': [{'spl_product_data_elements': ['KOLORZ SIXTY SECOND FLUORIDE CHERRY CHEESECAKE SODIUM FLUORIDE SODIUM FLUORIDE FLUORIDE ION WATER SUCRALOSE XYLITOL COCAMIDOPROPYL BETAINE CHERRY CHEESECAKE KOLORZ SIXTY SECOND FLUORIDE PINA COLADA SODIUM FLUORIDE SODIUM FLUORIDE FLUORIDE ION WATER SUCRALOSE XYLITOL COCAMIDOPROPYL BETAINE PIÑA COLADA KOLORZ SIXTY SECOND FLUORIDE TRIPLE MINT SODIUM FLUORIDE SODIUM FLUORIDE FLUORIDE ION WATER SUCRALOSE XYLITOL COCAMIDOPROPYL BETAINE TRIPLE MINT KOLORZ SIXTY SECOND FLUORIDE BLUE RASPBERRY SODIUM FL

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
  'terms': 'https://open.fda.gov/terms/',
  'license': 'https://open.fda.gov/license/',
  'last_updated': '2024-11-01',
  'results': {'skip': 0, 'limit': 1, 'total': 21}},
 'results': [{'spl_product_data_elements': ['KOLORZ SIXTY SECOND FLUORIDE CHERRY CHEESECAKE SODIUM FLUORIDE SODIUM FLUORIDE FLUORIDE ION WATER SUCRALOSE XYLITOL COCAMIDOPROPYL BETAINE CHERRY CHEESECAKE KOLORZ SIXTY SECOND FLUORIDE PINA COLADA SODIUM FLUORIDE SODIUM FLUORIDE FLUORIDE ION WATER SUCRALOSE XYLITOL COCAMIDOPROPYL BETAINE PIÑA COLADA KOLORZ SIXTY SECOND FLUORIDE TRIPLE MINT SODIUM FLUORIDE SODIUM FLUORIDE FLUORIDE ION WATER SUCRALOSE XYLITOL COCAMIDOPROPYL BETAINE TRIPLE MINT KOLORZ SIXTY SECOND FLUORIDE BLUE RASPBERRY 

In [53]:

# def extract_medication_data(result):
#     return {
#         'id': result['results'][0]['id'],
#         'effective_time': format_date(result['results'][0]['effective_time']),
#         'last_updated': result['meta']['last_updated'],

#         'brand_name': result['results'][0]['openfda'].get('brand_name', [None])[0],
#         'generic_name': result['results'][0]['openfda'].get('generic_name', [None])[0],
#         'manufacturer_name': result['results'][0]['openfda'].get('manufacturer_name', [None])[0],
#         'product_ndc': result['results'][0]['openfda'].get('product_ndc', [None])[0],
#         'package_ndc': result['results'][0]['openfda'].get('package_ndc', [None]),#[0],
#         'product_type': result['results'][0]['openfda'].get('product_type', [None])[0],
#         'route': result['results'][0]['openfda'].get('route', [None])[0],
#         'substance_name': result['results'][0]['openfda'].get('substance_name', [None])[0],
#         'rxcui': result['results'][0]['openfda'].get('rxcui', [None])[0],
#         'unii': result['results'][0]['openfda'].get('unii', [None]),#[0],

#         'elements': result['results'][0]['spl_product_data_elements'][0],
#         'indications_and_usage': result['results'][0]['indications_and_usage'][0],
#         'dosage_and_administration': result['results'][0]['dosage_and_administration'][0],
#         'description': result['results'][0]['description'][0],
#         'warnings': result['results'][0]['warnings'][0]
#     }


In [62]:

def safe_get(dictionary, keys, default=None):
    for key in keys:
        try:
            dictionary = dictionary[key]
        except (KeyError, IndexError, TypeError):
            return default
    return dictionary

def extract_medication_data(result):

    return {
        'id': safe_get(result, ['results', 0, 'id'], None),
        'code': safe_get(result, ['results', 0, 'openfda', 'rxcui'], [None])[0],
        'effective_time': format_date(safe_get(result, ['results', 0, 'effective_time'], None)),
        'last_updated': safe_get(result, ['meta', 'last_updated'], None),

        'brand_name': safe_get(result, ['results', 0, 'openfda', 'brand_name'], [None])[0],
        'generic_name': safe_get(result, ['results', 0, 'openfda', 'generic_name'], [None])[0],
        'manufacturer_name': safe_get(result, ['results', 0, 'openfda', 'manufacturer_name'], [None])[0],
        'product_ndc': safe_get(result, ['results', 0, 'openfda', 'product_ndc'], [None]),#[0], # pass the list
        'package_ndc': safe_get(result, ['results', 0, 'openfda', 'package_ndc'], None),
        'product_type': safe_get(result, ['results', 0, 'openfda', 'product_type'], [None])[0],
        'route': safe_get(result, ['results', 0, 'openfda', 'route'], [None])[0],
        'substance_name': safe_get(result, ['results', 0, 'openfda', 'substance_name'], [None])[0],
        'rxcui': safe_get(result, ['results', 0, 'openfda', 'rxcui'], [None]),#[0], # pass the list
        'unii': safe_get(result, ['results', 0, 'openfda', 'unii'], None),

        'elements': safe_get(result, ['results', 0, 'spl_product_data_elements'], [None])[0],
        'indications_and_usage': safe_get(result, ['results', 0, 'indications_and_usage'], [None])[0],
        'dosage_and_administration': safe_get(result, ['results', 0, 'dosage_and_administration'], [None])[0],
        'description': safe_get(result, ['results', 0, 'description'], [None])[0],
        'warnings': safe_get(result, ['results', 0, 'warnings'], [None])[0]
    }


In [55]:

def generate_medication_df(FHIR_product_codes: list[str])-> pd.DataFrame:
    medications = []
    for code in FHIR_product_codes:
        response = search_fda_drugs(code)
        medication = extract_medication_data(response)
        medications.append(medication)

    medications_df = pd.DataFrame(medications)
    print(medications_df.head())
    return medications_df


In [ ]:
medications_df = generate_medication_df([FHIR_product_code])
medications_df

---

# integrate extraction, transformation, and querying

In [54]:

# TODO 
# for every medication with status is active or not stopped
# nested elements in medication > ARRAY


In [60]:

def get_active_medications(medication_requests_df):
    active_statuses = ['active',]
    active_medications = medication_requests_df[medication_requests_df['status'].isin(active_statuses)]
    
    drug_codes = active_medications['code'].tolist()
    medications_df = generate_medication_df(drug_codes)
    return medications_df


In [63]:

active_medications_df = get_active_medications(medication_requests_df)
# print(active_medications_df)
active_medications_df

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.', 'terms': 'https://open.fda.gov/terms/', 'license': 'https://open.fda.gov/license/', 'last_updated': '2024-11-01', 'results': {'skip': 0, 'limit': 1, 'total': 43}}, 'results': [{'spl_product_data_elements': ['Diazepam Diazepam ANHYDROUS LACTOSE MAGNESIUM STEARATE MICROCRYSTALLINE CELLULOSE D&C YELLOW NO. 10 DIAZEPAM DIAZEPAM MYX;942;5 Chemical Structure'], 'spl_unclassified_section': ['Rx only', 'Medication Guides available at products.maynepharma.com or call 1-844-825-8500. Manufactured by: Mayne Pharma Greenville, NC 27834 Revised: 03/2021 61771'], 'boxed_warning': ["WARNINING: RISKS FROM CONCOMITANT USE WITH OPIOIDS; ABUSE, MISUSE, AND ADDICTION; and DEPENDENCE AND WITHDRAWAL REACTIONS Concomitant

,id,code,effective_time,last_updated,brand_name,generic_name,manufacturer_name,product_ndc,package_ndc,product_type,route,substance_name,rxcui,unii,elements,indications_and_usage,dosage_and_administration,description,warnings
0,a53f82fa-2b99-4989-b4ca-fa931142443e,197591,2024-08-07,2024-11-01,Diazepam,DIAZEPAM,Bryant Ranch Prepack,[71335-2197],"[71335-2197-1, 71335-2197-2, 71335-2197-3, 713...",HUMAN PRESCRIPTION DRUG,ORAL,DIAZEPAM,[197591],[Q3JTX2Q7TU],Diazepam Diazepam ANHYDROUS LACTOSE MAGNESIUM ...,INDICATIONS Diazepam Tablets USP are indicated...,DOSAGE AND ADMINISTRATION Dosage should be ind...,DESCRIPTION Diazepam is a benzodiazepine deriv...,WARNINGS Risks from Concomitant Use with Opioi...
1,9a63ba0a-0df9-744b-e053-2a95a90a9344,310325,2019-12-23,2024-11-01,Ferrous Sulfate,FERROUS SULFATE,Richmond Pharmaceuticals Inc.,[54738-963],[54738-963-13],HUMAN OTC DRUG,ORAL,FERROUS SULFATE,[310325],[39R4TAN1VT],Ferrous Sulfate Ferrous Sulfate CROSCARMELLOSE...,SUGGESTED USE:,DRUG INTERACTION PRECAUTION: Since oral iron p...,None,WARNINGS: Do not exceed recommended dosage. Th...


In [ ]:
# active_medications_df['rxcui'].apply(lambda x: x[0] if x else None)
# active_medications_df['package_ndc'].apply(lambda x: x[0] if x else None)


In [66]:
active_medications_df.to_csv('active_medications.csv', index=False)

---

# export to csv/db-ready format

In [69]:

def export_to_csv(df, file_path):
    df.to_csv(file_path, index=False)


In [100]:
patients, medications, encounters = process_json_files(DATA_DIRECTORY)

patients_df = pd.DataFrame(patients)
medication_requests_df = pd.DataFrame(medications)
encounters_df = pd.DataFrame(encounters)


directory:  c:\locr\mendel\data\


In [103]:
f'{active_medications_df=}'.split('=')[0]

'active_medications_df'

In [108]:

df_names = ['active_medications_df', 'medication_requests_df', 'patients_df', 'encounters_df']
df_list = [active_medications_df, medication_requests_df, patients_df, encounters_df]

for df_name, df in zip(df_names, df_list):
    print(df_name)
    file_path = OUT_DIRECTORY+df_name+'.csv'
    print('file_path: ', file_path)
    export_to_csv(df, file_path)


active_medications_df
file_path:  c:\locr\mendel\out\active_medications_df.csv
medication_requests_df
file_path:  c:\locr\mendel\out\medication_requests_df.csv
patients_df
file_path:  c:\locr\mendel\out\patients_df.csv
encounters_df
file_path:  c:\locr\mendel\out\encounters_df.csv


---
